In [4]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from numpy import linspace
import argparse
import cv2 as cv
import glob
from PIL import Image

Hs = np.load("run/numpy/Hs.npy")

def getK(Hs):
    V = np.zeros((2*len(Hs),6))
    for idx, H in enumerate(Hs):
        h1, h2 = H[:,0], H[:,1]

        # h1^T*B*h2 = 0
        v1 = [h1[0]*h2[0],                  # h10h20
            h1[1]*h2[0]+h1[0]*h2[1],        # h11h20+h10h21
            h1[2]*h2[0]+h1[0]*h2[2],        # h12h20+h10h22
            h1[1]*h2[1],                    # h11h21
            h1[2]*h2[1]+h1[1]*h2[2],        # h12h21+h11h22
            h1[2]*h2[2]                     # h12h22
        ]
        # h1^T*B*h1 = h2^T*B*h2
        v2 = [h1[0]**2-h2[0]**2,            # h10^2 - h20^2
            2*(h1[0]*h1[1]-h2[0]*h2[1]),    # 2(h10h11-h20h21)
            2*(h1[0]*h1[2]-h2[0]*h2[2]),    # 2(h10h12-h20h22)
            h1[1]**2-h2[1]**2,              # h11^2-h21^2
            2*(h1[1]*h1[2]-h2[1]*h2[2]),    # 2(h11h12-h21h22)
            h1[2]**2-h2[2]**2]              # h12^2-h22^2

        # put v1 and v2 into V
        V[2*idx] = np.array(v1)
        V[2*idx+1] = np.array(v2)

    # solve b = argmin(Vb)
    U, D, VT = np.linalg.svd(V)
    b11, b12, b13, b22, b23, b33 = VT[-1]

    # reconstruct B from b
    B = np.array([[b11,b12,b13],[b12,b22,b23],[b13,b23,b33]])

    # make it positive definite (if it isn't yet)
    if(B[0][0] < 0):
        B = -B

    # B = L*L^T
    L = np.linalg.cholesky(B)

    # L = K^-1^T
    K = np.linalg.inv(np.transpose(L))

    # normalize
    K /= K[2][2]

    K[0][1] = 0

    print(K)
    return K
K=getK(Hs)
np.save('drone_calib_mtx.npy',K)

[[1.18162341e+03 0.00000000e+00 4.79134393e+02]
 [0.00000000e+00 6.86457797e+02 8.79928518e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
